In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import seaborn as sns
import numpy as np
from datetime import datetime
import statsmodels.formula.api as smf
import statsmodels.api as sm
from statsmodels.formula.api import ols
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures
from sklearn.metrics import r2_score
from scipy import stats
import math
from PIL import Image
from sklearn import metrics
import pickle
from statsmodels.stats.outliers_influence import variance_inflation_factor


Functions

In [ ]:
#Cleaning Data
def fill_na(df):
    df.fillna(df.median(), inplace= True)

    
#Preparing data for use
def date_prep(df):
    df['year_sold'] = df['date'].map(lambda x: x[-4:])
    df['month'] = df['date'].map(lambda x: x[0:2])
    df['month'] = df['month'].map(lambda x: x.replace('/',''))
    df['month'] = df['month'].astype(int)
    df['year_sold'] = df['year_sold'].astype(int)

#Binning Data
def date_bins(df):    
    bins = [0, 3, 6, 9, 12]
    date_bins = pd.cut(df['month'], bins, include_lowest=True, labels=['Winter', 'Spring', 'Summer', 'Fall'])
    date_bins = date_bins.cat.as_unordered()
    df["season"] = date_bins
    df = df.drop(["month","date"], axis = 1, inplace= True)

#Function to clean data
def data_cleaning(df):
    df = df[(df["price"] > 100000) & (df["price"]< 1500000)]
    df = df[(np.abs(stats.zscore(df["sqft_living"])) < 3)]
    df = (df[df["bedrooms"] < 7])
    df = (df[df["bathrooms"] < 6])
    df = (df[df["sqft_living"] < 8000])
        
#Function to convert categorical data 
# def convert_categorical(column_names):
#     categorical_encoder = preprocessing.OneHotEncoder(handle_unknown= "ignore")
#     cat_df = categorical_encoder.fit_transform([column_names])

#OLS regression function    
def OLS_reg(df):
    y = df["price"]
    X = df.drop("price", axis=1)
    model = sm.OLS(y, sm.add_constant(X), missing = "drop").fit()
    results = model.summary()
    print(results)

#Predrictive model function
def predictive_model(df):
    df2 = df.to_numpy()
    y = df2[:, 0]
    X = df2[:,1:]
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    data_transformation = preprocessing.StandardScaler()
    data = data_transformation.fit_transform(X_train)
    model = LinearRegression().fit(data, y_train)
    accuracy = model.score(data, y_train)
    test_accuracy = model.score(data_transformation.transform((X_test)), y_test)#change accuracy
    y_pred = model.predict(data_transformation.transform(X_test))
    print("X_train shape", X_train.shape)
    print("X_test shape",X_test.shape)
    print("y_train shape",y_train.shape)
    print("y_test shape",y_test.shape)
    print("R^2",accuracy)
    print("Test R^2",test_accuracy)
    print('Mean Absolute Error:', metrics.mean_absolute_error(y_test, y_pred))
    print('Mean Squared Error:', metrics.mean_squared_error(y_test, y_pred))
    print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y_test, y_pred)))
    model1 = pd.DataFrame({'Actual value': y_test, 'Predicted value': (y_pred.round(2)), "Difference" : (abs(y_test-y_pred)).round(2)})
    print(model1.sample(10).head(10))
    true_value = model1["Actual value"]
    predicted_value = model1["Predicted value"]
    plt.figure(figsize=(20,20))
    plt.scatter(true_value, predicted_value, c='crimson')
    # plt.yscale('log')
    # plt.xscale('log')
    p1 = min(min(predicted_value), min(true_value))
    p2 = max(max(predicted_value), max(true_value))
    plt.plot([p1, p2], [p1, p2], 'b-')
    plt.xlabel('Predictions''True Values', fontsize=15)
    plt.ylabel('True Values', fontsize=15)
    plt.axis('equal')
    plt.show()

#Prediction model for logged data
def predictive_model_log(df):
    df2 = df.to_numpy()
    y = df2[:, 0]
    X = df2[:,1:]
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    scaler  = preprocessing.StandardScaler()
    data = scaler.fit_transform(X_train)
    model = LinearRegression().fit(data, y_train)
    accuracy = model.score(data, y_train)
    test_accuracy = model.score(scaler.transform((X_test)), y_test)#change accuracy
    y_pred = model.predict(scaler.transform(X_test))
    y_pred = np.exp(y_pred)
    y_test = np.exp(y_test)
    print("X_train shape", X_train.shape)
    print("X_test shape",X_test.shape)
    print("y_train shape",y_train.shape)
    print("y_test shape",y_test.shape)
    print("R^2",accuracy)
    print("Test R^2",test_accuracy)
    print('Mean Absolute Error:', metrics.mean_absolute_error(y_test, y_pred))
    print('Mean Squared Error:', metrics.mean_squared_error(y_test, y_pred))
    print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y_test, y_pred)))
    model1 = pd.DataFrame({'Actual value': y_test, 'Predicted value': (y_pred.round(2)), "Difference" : (abs(y_test-y_pred)).round(2)})
    print(model1.sample(10).head(10))
    true_value = model1["Actual value"]
    predicted_value = model1["Predicted value"]
    plt.figure(figsize=(20,20))
    plt.scatter(true_value, predicted_value, c='crimson')
    plt.yscale('log')
    plt.xscale('log')
    p1 = min(min(predicted_value), min(true_value))
    p2 = max(max(predicted_value), max(true_value))
    plt.plot([p1, p2], [p1, p2], 'b-')
    plt.xlabel('Predictions''True Values', fontsize=15)
    plt.ylabel('True Values', fontsize=15)
    plt.axis('equal')
    plt.show()

#Polynomial prediction function
def poly_model(df):
    df2 = df.to_numpy()
    y = df2[:, 0]
    X = df2[:,1:]
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    data_transformation = preprocessing.StandardScaler()
    data = data_transformation.fit_transform(X_train)
    model = LinearRegression().fit(data, y_train)
    for index, degree in enumerate([2,3]):  
    # Instantiate PolynomialFeatures
        poly = PolynomialFeatures(degree)
    # Fit and transform X_train
        X_poly_train = poly.fit_transform(X_train)
    # Instantiate and fit a linear regression model to the polynomial transformed train features
        reg_poly = LinearRegression().fit(X_poly_train, y_train)
    # Transform the test data into polynomial features
        X_poly_test = poly.transform(X_test)
    # Get predicted values for transformed polynomial test data  
        y_pred = reg_poly.predict(X_poly_test)
    # Evaluate model performance on test data
        print("Train degree %d" % degree, reg_poly.score(X_poly_train, y_train))
        print("Test degree %d" % degree, reg_poly.score(X_poly_test, y_test))
        print("train degree %d" % degree, reg_poly.score(X_poly_train, y_train))

#Polynomial prediction function for logged data
def poly_model_log(df):
    df2 = df.to_numpy()
    y = df2[:, 0]
    X = df2[:,1:]
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    data_transformation = preprocessing.StandardScaler()
    data = data_transformation.fit_transform(X_train)
    model = LinearRegression().fit(data, y_train)
    for index, degree in enumerate([2,3]):  
    # Instantiate PolynomialFeatures
        poly = PolynomialFeatures(degree)
    # Fit and transform X_train
        X_poly_train = poly.fit_transform(X_train)
    # Instantiate and fit a linear regression model to the polynomial transformed train features
        reg_poly = LinearRegression().fit(X_poly_train, y_train)
    # Transform the test data into polynomial features
        X_poly_test = poly.transform(X_test)
    # Get predicted values for transformed polynomial test data  
        y_pred = reg_poly.predict(X_poly_test)
        y_pred = np.exp(y_pred)
        y_test = np.exp(y_test)
    # Evaluate model performance on test data
        print("Train degree %d" % degree, reg_poly.score(X_poly_train, y_train))
        print("Test degree %d" % degree, reg_poly.score(X_poly_test, y_test))
        print("train degree %d" % degree, reg_poly.score(X_poly_train, y_train))

In [ ]:
#Importing Data
#Read dataset
pd.set_option('display.max_columns', None)
df1 = pd.read_csv("data/kc_house_data.csv", index_col="id")
df_income = pd.read_csv("data/dc_housing_income_by_zip.csv")#source https://www.kaggle.com/miker400/washington-state-home-mortgage-hdma2016?select=Washington_State_HDMA-2016.csv


In [ ]:
#Using median income per zipcode
income = df_income.groupby("zipcode").median().round()

In [ ]:
#Keeping last value in dataset
df1 = df1[~df1.index.duplicated(keep='last')]

In [ ]:
#Merging datasets on zipcode
df = df1.merge(income, on ="zipcode")

In [ ]:
#Exploring data
df

In [ ]:
#Running functions to change date to usable format and bin dates into seasons
date_prep(df)
date_bins(df)

In [ ]:
df

In [ ]:
#Changes to split the grade and extract the number and dropping original grade column
new = df["grade"].str.split(" ", n = 1, expand = True)
df["grade_1"]=new[0]
df.drop("grade",axis=1, inplace=True)

In [ ]:
#Checking value counts in view
df['view'].value_counts()

In [ ]:
#Updating view from rating to number 
df['view'] = df['view'].map({'NONE': 1, 'AVERAGE': 2, 'GOOD': 3, 'FAIR': 4, 'EXCELLENT': 5})

In [ ]:
#Checking valuecounts in waterfron
df['waterfront'].value_counts()

In [ ]:
#Updating waterfron to number
df['waterfront'] = df['waterfront'].map({'YES': 1, 'NO': 0})

In [ ]:
#Checking value counts in condition
df['condition'].value_counts()

In [ ]:
#Updating condition to number
df['condition'] = df['condition'].map({'Poor': 1, 'Fair': 2, 'Average': 3, 'Good': 4, 'Very Good': 5})

In [ ]:
#Checking value counts in season
df['season'].value_counts()

In [ ]:
#Chaning season to number
df['season'] = df['season'].map({'Spring': 1, 'Summer': 2, 'Fall': 3, 'Winter': 4})

In [ ]:
#Checking for NAN values
df.isna().sum()

In [ ]:
#Replacing missing values with median values of each column
df['waterfront'].fillna((df['waterfront'].median()), inplace=True)
df['grade_1'].fillna((df['grade_1'].median()), inplace=True)
df['yr_renovated'].fillna((df['yr_renovated'].median()), inplace=True)
df['view'].fillna((df['view'].median()), inplace=True)

In [ ]:
#Checking NAN values
df.isna().sum()

In [ ]:
#Checking variable types
df.dtypes

In [ ]:
#Converting variables to floats 
df["sqft_basement"] = df["sqft_basement"].replace("?","0").astype(float)
df["season"] = df["season"].astype(float)
df["grade_1"] = df["grade_1"].astype(float)

In [ ]:
#Checking correlation between variables
initial_corr = df.corr().sort_values(by="price", ascending=False)
plt.figure(figsize=(10,10))
ax = sns.heatmap(initial_corr, linewidth=0.5)
plt.show()
ax.set_title("Heatmap of Correlation Between Attributes (Including Target)");

In [ ]:
#Model for highest correlation
high_corr_formula = 'price ~ sqft_living'
high_corr_formula = ols(formula=high_corr_formula, data=df).fit()
singular_model = high_corr_formula.summary()
print(singular_model)

In [ ]:
#First OLS model
OLS_reg(df)

In [ ]:
#First Predictive model
predictive_model(df)

In [ ]:
#Plotting data to check for correlation and outliers
sns.pairplot(df, y_vars="price")

As we can see from the charts there are some outliers that need to be addressed, after the outliers are addressed we will also need to log tranform some of our data to turn into normal distribution

In [ ]:
#Checking data for outliers
#Check price
#Check bedroom 
#Check sqft_living
#Check sqft_lot15

df.describe()

In [ ]:
df.boxplot('price', vert=False, figsize=(20,10))

In [ ]:
df = df[df["price"] < 4000000]

In [ ]:
df.boxplot('sqft_living', vert=False, figsize=(20,10))

In [ ]:
#Dropping outliers from sqft_living
df = df = df[df["sqft_living"] < 8000]

In [ ]:
df.boxplot('bedrooms', vert=False, figsize=(20,10))

In [ ]:
df.describe()

In [ ]:
#Replacing 33 with 3 as it seems to be a typo
df["bedrooms"] = df["bedrooms"].replace(33,3)

In [ ]:
df.boxplot('bedrooms', vert=False, figsize=(20,10))

In [ ]:
df["bedrooms"].value_counts()

In [ ]:
#Dropping houses above 9 bedrooms
df = (df[df["bedrooms"] < 9])

In [ ]:
#Checking for multicollinearity  
corr = df.corr()
plt.figure(figsize=(15,15))
ax = sns.heatmap(corr, linewidth=0.5,annot = True)
plt.show()
ax.set_title("Heatmap of Correlation Between Attributes (Including Target)");


In [ ]:
df.shape
#High Colinearity to price : sqft living, grade, bathrooms, view, lat, bedrooms, basement, floors, waterfront
#sqft_living:sqft above, sqftliving15,bathrooms(possibly )
#bathrooms
#view
#waterfront

In [ ]:
df_categorical = df[["view","waterfront"]]

In [ ]:
df_cat =pd.get_dummies(df_categorical, columns=["view","waterfront"], drop_first= True) 
df_cat

In [ ]:
df_numerical = df[["price", "sqft_living","bathrooms"]]

In [ ]:
df = pd.concat([df_numerical, df_cat], axis=1)
df

In [ ]:
#Model with outliers removed
OLS_reg(df)

In [ ]:
#Predictive model with outliers removed
predictive_model(df)

In [ ]:
df_numerical = df[["price", "sqft_living","bathrooms"]].copy().apply(lambda x: np.log(abs(x+1)))

In [ ]:
df_cat

In [ ]:
log_df = pd.concat([df_numerical, df_cat], axis=1)

In [ ]:
log_df

In [ ]:
#Log transformed 
OLS_reg(log_df)

In [ ]:
#Running predictive model for logged data
predictive_model_log(log_df)

#Running OLS regression on selected features 
OLS_reg(log_df)

In [ ]:
#Running prediction model on selected features
predictive_model(log_df)

In [ ]:
#Running polynomial function for selected features 
poly_model(log_df)